# Include Libraries 

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as k 

import matplotlib.pyplot as plt 
import numpy as np
from sklearn.metrics import classification_report
from keras.optimizers import SGD 
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import argparse
ap = argparse.ArgumentParser()
args = vars(ap.parse_args())

# Load Data

In [11]:
((train_images, train_labels),(test_images, test_labels)) = cifar10.load_data()
labelNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
test_images.shape

(10000, 32, 32, 3)

In [12]:
x = np.concatenate((train_images, test_images))
y = np.concatenate((train_labels, test_labels))

train_images, test_images, train_labels, test_labels = train_test_split(x, y, test_size=0.1, random_state=42)
train_images = train_images.reshape((54000, 32 , 32, 3))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((6000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

# One Hot Encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Model Architecture

In [13]:
class MiniVGG:
    def build(width, height, depth, classes):
        
        model = Sequential()
        inputshape = (height, width, depth)
        chanDim = -1
        
        if k.image_data_format() == "channels_first":
            inputshape = (depth, height, width)
            chanDim = 1
        # First Block
        model.add(Conv2D(32, (3,3), padding = 'SAME', input_shape = inputshape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis =chanDim))
        model.add(Conv2D(32, (3,3), padding = 'SAME'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis =chanDim))
        model.add(MaxPooling2D(pool_size = (2,2))) # as we dont add stridr keras put it to be equal max pooling 
        model.add(Dropout(0.25)) # to reduce overfitting
        
        # Second Block
        model.add(Conv2D(64, (3,3), padding = 'SAME'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis =chanDim))
        model.add(Conv2D(64, (3,3), padding = 'SAME'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis =chanDim))
        model.add(MaxPooling2D(pool_size = (2,2)))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5)) # between Layers
        
        # Classifier
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [ ]:
# Compile Model

In [ ]:
print("[INFO] Compiling Model")

opt = SGD(lr= 0.01, decay = 0.01/ 40, momentum = 0.9, nesterov =True)
# alpha = 0.01, Beta = 0.9 , decay rate of learning rate is (0.01/40)
# nesetrov = true mean that we wold lik to apply nesetrov acclerated Gradient

model = MiniVGG.build(width = 32, height = 32, depth = 3, classes =10) # instantiate Model 
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, validation_data = (test_images, test_labels), epochs=40, batch_size=64
         verbose =1)

[INFO] Compiling Model
Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 780s 14ms/step - loss: 1.6063 - accuracy: 0.4324 - val_loss: 1.1944 - val_accuracy: 0.5743
Epoch 2/5
54000/54000 [==============================] - 747s 14ms/step - loss: 1.1924 - accuracy: 0.5764 - val_loss: 1.2486 - val_accuracy: 0.5800
Epoch 3/5
38016/54000 [====================>.........] - ETA: 3:50 - loss: 1.0133 - accuracy: 0.6440

# Evaluate the network 

In [ ]:
print("[INFO] evaluating network")
predictions = model.predict(test_images, batch_size = 64)
print(classification_report(test_labels.argmax(axis = 1),
                           predictions.argmax(axis = 1), target_names = labelNames)

# Plotting the Network 

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,40), H.history["loss"], label = ["train_loss"])
plt.plot(np.arange(0,40), H.history["val_loss"], label = ["val_loss"])
plt.plot(np.arange(0,40), H.history["acc"], label = ["train_acc"])
plt.plot(np.arange(0,40), H.history["val_acc"], label = ["val_acc"])

plt.title("Training loss and Accuracy on CIFAR-10")
plt.xlabel("EPOCH #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])